# Derivation

\begin{equation}
    
\end{equation}

# Code

In [22]:
# first create the necessary functions

# get angle for two given turbines
function get_angle(dx, dy)
    angle = atan(dy,dx)
end

# get probability for a given angle
function get_probability(angle, wind_rose)
    p = 1.0
end

# get effective velocity for a given turbine
function get_effvelocity(turbid, x, y, wind_rose, uave; r0=0.5, alpha=0.1, a=1.0/3.0)
    nturbines = length(x)
    deficit = 0.0
    for j = 1:nturbines
        dx = x[j] - x[turbid]
        dy = y[j] - y[turbid]
        dr = sqrt(dx^2+dy^2)
        angle = get_angle(dx, dy)
        p = get_probability(angle, wind_rose)
        deficit += (r0/(r0+alpha*dr))^4
    end
    deficit = sqrt((4.0*a^2)*deficit)
    
    ueff = uave - deficit
    println("ueff ", ueff)
    return ueff
end

# get effective cp for a turbine in a given direction
function get_cp(u, cp_dist)
    cp = 0.6
end

# get average wind speed
function get_ave_wind_speed(wind_rose)
    wind_speeds = wind_rose[:,2]
    probabilities = wind_rose[:,3]
    ave_speed = sum(wind_speeds.*probabilities)
    return ave_speed
end

# get average power for a given turbine
function get_effpower(turbid, x, y, wind_rose, uave, cp_dist;prated=2E6, u_cut_in=3.0, u_cut_out=15.0, density=1.225, r0=0.5, alpha=0.1, a=1.0/3.0)
    area = pi*r0^2
    ueff = get_effvelocity(turbid, x, y, wind_rose, uave)
    peff = 0.0
    
    if ueff < u_cut_in
        return peff
    elseif ueff > u_cut_out
        return peff
    else 
        cp = get_cp(ueff, cp_dist)
        peff = 0.5*density*area*cp*ueff^3
    end
        
    if peff > prated
        peff = prated
    end
    println("peff ", peff)
    return peff
end

# get annual energy production
function get_aep(x, y, wind_rose, cp_dist; density=1.225, r0=0.5, alpha=0.1, a=1.0/3.0)
    nturbines = length(x)
    uave = get_ave_wind_speed(wind_rose)
    power = zeros(nturbines)
    for i = 1:nturbines
        power[i] = get_effpower(i, x, y, wind_rose, uave, cp_dist)
    end
    aep = (365.0)*(24.0)*sum(power)
    return aep
end

get_aep (generic function with 1 method)

In [25]:
x = [0.0, 3.0, 7.0]
y = [0.0, 0.0, 0.0]

wind_rose = [0.0 10.0 0.5; pi 10.0 0.5]
print(wind_rose)
cp_dist = [10.0, 0.6]
aep = get_aep(x,y,wind_rose,cp_dist)

[0.0 10.0 0.5; 3.141592653589793 10.0 0.5]ueff 9.274977790830214
peff 230.29571634159396
ueff 9.255285958072967
peff 228.83199639203715
ueff 9.292767306303833
peff 231.62338918251388


6.05097965278543e6

In [26]:
365*24*2E6


1.752e10